In [3]:
#!pip install dash dash_core_components dash_html_components

In [13]:
import dash
from dash import dcc
from dash import html
from dash import dash_table
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd

In [14]:
# Load the data
df = pd.read_csv('ycsb_report.csv')

#Convert to string 
df['Operations'] = df['Operations'].apply(lambda x: "10k" if x==10000 else ("100k" if x==100000 else "1M"))


# Define workload descriptions
workload_descriptions = {
    "a": "Workload A Update heavy workload: 50% read, 50% update operations",
    "b": "Workload B Read mostly workload: 95% read, 5% update operations.",
    "c": "Workload C Read only: 100% read operations.",
    "d": "Workload D Read latest workload: New records are read more frequently.",
    "e": "Workload E Short ranges: Range queries over the latest records.",
    "f": "Workload F Read-modify-write: Client read a record, modifies it, and writes back the changes.",
}

# Define the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    dcc.Dropdown(
        id='workload_dropdown',
        options=[{'label': workload_descriptions[wl], 'value': wl} for wl in df['WorkloadType'].unique()],
        value=df['WorkloadType'].unique()[0]
    ),
    dcc.Graph(
        id='comparison_graph'
    ),
    dcc.Graph(
        id='bar_plot'
    ),
    dcc.Graph(
        id='heatmap_plot'
    ),
    dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i} for i in df.describe().columns],
        data=df.describe().to_dict('records')
    )
])

# Define the callback to update the graph
@app.callback(
    Output('comparison_graph', 'figure'),
    Input('workload_dropdown', 'value'))
def update_figure(selected_wl):
    filtered_df = df[df['WorkloadType'] == selected_wl]
    fig = px.line(filtered_df,
                  markers=True,
                  x='Operations',
                  y='AverageLatency(us)',
                  color='Database',
                  title='Average Latency (us) / Operations for WorkloadType ' + selected_wl)
    return fig

# Bar plot
@app.callback(
    Output('bar_plot', 'figure'),
    Input('workload_dropdown', 'value'))
def update_bar(selected_wl):
    filtered_df = df[df['WorkloadType'] == selected_wl]
    fig = px.bar(filtered_df, x='Database', y='AverageLatency(us)', color='Operations', barmode='group', title='AverageLatency(us) per Database for WorkloadType '+selected_wl)
    return fig

# Heat map
@app.callback(
    Output('heatmap_plot', 'figure'),
    Input('workload_dropdown', 'value'))
def update_heatmap(selected_wl):
    filtered_df = df[df['WorkloadType'] == selected_wl]
    stats = filtered_df.groupby('Database')['AverageLatency(us)'].describe()
    fig = go.Figure(data=go.Heatmap(
        z=stats.values,
        x=stats.columns,
        y=stats.index,
        hoverongaps = False))
    fig.update_layout(title_text='Statistics of AverageLatency(us) per Database for WorkloadType '+selected_wl)
    return fig

if __name__ == '__main__':
    app.run_server(debug=False)


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [21/May/2023 19:42:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2023 19:42:06] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2023 19:42:06] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2023 19:42:06] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2023 19:42:06] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2023 19:42:06] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2023 19:42:06] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2023 19:42:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2023 19:42:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2023 19:42:07] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 200 -
127.0.0.1 